# Solving the 1-D Diffusion Equation (Explicit)

## The Problem

We are interested in how the temperature $T$ changes with time $t$, for a given initial temperature $T_{0}[x,t_{0}]$ and boundary conditions; e.g., a dike intrusion in the lithosphere, with 
- a total length $L$ = 100 m, 
- a dike width $W$ = 5 m, 
- a thermal diffusivity $\kappa$ = 10⁻⁶ m²/s, 
- a dike temperature $T_{dike}$ = 1200 °C, and 
- a background temperature $T_{background}$ = 300 °C.

<img src="../Figures/Exercise02_1b.png" alt="drawing" width="350"/> <br>
**Fig. 1.** Sketch of the geological problem and the profile of the initial temperature condition.

How long would it take for the dike to cool to a given temperature? How does the solution behave over time? Is the temperature of the host rock important? How does the solution depend on the chosen numerical parameters (e.g., the number of grid points or the time step size)?

To determine the cooling time, we need to solve the temperature diffusion equation (a parabolic partial differential equation (PDE)):

$$
\rho c_p \frac{\partial{T}}{\partial{t}} = \frac{\partial{}}{\partial{x}} \left( k\frac{\partial{T}}{\partial{x}} \right), \tag{1}
$$

where $k$ is the thermal conductivity [W/m/K], $T$ is the temperature [K], and $\rho$ is the density [kg/m³].

Assuming the thermal parameters are constant, we can rewrite the equation and express it using the thermal diffusivity [m²/s] $\kappa = \frac{k}{\rho c_p}$:

$$
\frac{\partial{T}}{\partial{t}} = \kappa \frac{\partial^2{T}}{\partial{x^2}}. \tag{2}
$$

## The Solution

### Finite Difference Approximation

To solve the problem numerically using finite differences, we first need to create a numerical grid (the discretization):

<img src="../Figures/Exercise02_2.png" alt="drawing" width="600"/> <br>
**Fig. 2.** Numerical 1-D grid for the discretization of the *PDE*.

To solve the diffusion equation numerically using the explicit finite difference scheme, we now need to rewrite the *PDE* using finite difference expressions.

Forward in time (FT):
$$
\frac{\partial{T}}{\partial{t}} \approx \frac{T_{I^\textrm{C}}^{n+1}-T_{I^\textrm{C}}^{n}}{t^{n+1}-t^{n}} = \frac{T_{I^\textrm{C}}^{n+1}-T_{I^\textrm{C}}^{n}}{\Delta t} \tag{3}
$$

Central in space (CS):
$$
\frac{\partial^2{T}}{\partial{x^2}} \approx \frac{\frac{T_{I^\textrm{E}}^{n}-T_{I^\textrm{C}}^{n}}{\Delta x} - \frac{T_{I^\textrm{C}}^{n}-T_{I^\textrm{W}}^{n}}{\Delta x}}{\Delta x} =
\frac{T_{I^\textrm{E}}^{n} - 2T_{I^\textrm{C}}^{n} + T_{I^\textrm{W}}^{n}}{(\Delta x)^2} \tag{4}
$$

Thus, in explicit form, the temperature at each point at the new time is given by:
$$
T_{I^\textrm{C}}^{n+1} = T_{I^\textrm{C}}^{n} + \kappa \Delta t \left( \frac{T_{I^\textrm{E}}^{n}-2T_{I^\textrm{C}}^{n}+T_{I^\textrm{W}}^{n}}{\Delta x^2} \right) \tag{5}
$$


Let us now first load the necessary modules for the numerical solution of our problem. We will begin by programming the solvers ourselves, before making use of the predefined solvers in `GeoModBox.jl`. The predefined solver is located in the submodule `GeoModBox.HeatEquation.OneD`.

In [ ]:
using Plots
using GeoModBox.HeatEquation.OneD
a = time()

### Parameter Definitions

Let us first define the parameters for the problem (physical constants, numerical domain, and time parameters).

The parameter `alternative` describes different programming approaches:

1. Fully explicit implementation (loop)
2. Fully explicit implementation (index array)
3. Use of the built-in function

In [ ]:
alternative =   3
# Physics --------------------------------------------------------------- #
L           =   100.0       # Length [m]
Tdike       =   1200.0      # Dike temperature [C]
Trock       =   300.0       # Background temperature [C]
κ           =   1.0e-6      # Thermal Diffusivity [m2/s]
W           =   5.0         # Dike width [m]
# ----------------------------------------------------------------------- #
# Numerical Parameter --------------------------------------------------- #
nc          =   100                 # Number of centroids 
Δx          =   L/(nc)              # Grid spacing 
xc          =   Δx/2:Δx:(L-Δx/2)    # x-coordinates of the centroids  
ind         =   1:nc                # Indexes for internal points
# ----------------------------------------------------------------------- #
# Time Parameter -------------------------------------------------------- #
day         =   3600.0*24.0             # Seconds per day
Δt          =   0.9 * Δx^2 / (2.0 * κ)
nt          =   floor(Int, 200.0 * day/Δt ) 
Time        =   0.0
# ----------------------------------------------------------------------- #

### Initial Conditions

To solve our problem, we first need to define the initial conditions. We assume 300 °C for the host rock and 1200 °C for the dike. That is, the initial temperature is defined by:

$$
T \left(x < \left( \frac{L}{2} - W \right), x > \left( \frac{L}{2} + W \right), t = 0 \right) = 300, \tag{6}
$$

$$
T \left(x > \left( \frac{L}{2} - W \right), x < \left( \frac{L}{2} + W \right), t = 0 \right) = 1200. \tag{7}
$$

Next, we will first visualize the initial condition graphically.

In [ ]:
# Initial Condition; Temperature Profile -------------------------------- #
T       =   (
    T       =   zeros(nc), 
    T_ex    =   zeros(nc+2),
)
T.T     .=  Trock                       # Background T
@. T.T[abs(xc-L/2) <= W/2] =  Tdike     # Dike T

T.T_ex[2:end-1]     .=  T.T

p = plot(xc, T.T, xlabel="x [m]", ylabel="T [°C]", 
            title="Temperature distribution after 
            $(round(Time/day, digits=1)) days", 
            xlim=(0,L), ylim=(0, Tdike))
display(p)
# ----------------------------------------------------------------------- #

### Boundary Conditions

Since we use cell-centered grid points (centroids) for the temperature, no grid point (vertex) lies directly on the boundaries (contrary to expectations, this is actually an advantage!). To prescribe the temperature boundary conditions, we therefore need to make use of additional *ghost nodes* (see Figure 2), i.e., we determine the temperature on the *ghost nodes* in order to be able to solve the temperature partial differential equation at the **nearest interior** grid point using finite differences.

For a constant temperature boundary condition at the boundaries (Dirichlet), we can determine the temperature on the *ghost nodes* by linear interpolation such that:

$$
T_{Ghost}^W = 2 T_{BC}^W - T_1 \tag{8}
$$

$$
T_{Ghost}^E = 2 T_{BC}^E - T_{nc} \tag{9}
$$

In the implementation, we make use of a small trick and define the temperature on the *ghost nodes* later in the script. For this purpose, we first define a *tuple* that contains the type and the value of the temperature boundary conditions.


In [ ]:
# Boundary Conditions --------------------------------------------------- #
BC          =   (
                    type = (W=:Dirichlet, E=:Dirichlet),
                    #type = (W=:Neumann, E=:Neumann),
                    val = (W=:300.0,E=:300.0)
)
# ----------------------------------------------------------------------- #

### Visualization

To visualize the results as an animation in a GIF file, we first need to specify the location and the name of the file:

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("02_1D_explicit_",alternative)
save_fig    =   1
# ----------------------------------------------------------------------- #

### Time Loop

Numerically, we can now solve the *ODE* in a time loop in different ways (Alternative I is sufficient; those interested can also implement Alternatives II and III). At each time step, the temperature at the ghost nodes is determined using the equations above.

In [ ]:
# Time loop ------------------------------------------------------------- #
for n = 1:nt
    println("Time Step: ",n,", Time: $(round(Time/day, digits=1)) [d]")        

    if alternative == 1       
        # Alternative I ---
        # Programming a loop over the grid (without boundary nodes)
        # Define the boundary conditions
        # West
        T.T_ex[1]    =   (BC.type.W==:Dirichlet) * (2 * BC.val.W - T.T_ex[2]) + 
                                (BC.type.W==:Neumann) * (T.T_ex[2] - BC.val.W*Δx)
        # East
        T.T_ex[end]  =   (BC.type.W==:Dirichlet) * (2 * BC.val.E - T.T_ex[nc+1]) +
                                (BC.type.W==:Neumann) * (T.T_ex[nc+1] + BC.val.E*Δx)
        for i = 1:nc
            # Calculate temperature at point i for the new time step
            T.T[i]    =   T.T_ex[i+1] + κ * Δt * 
                                    (T.T_ex[i + 2] - 2.0 * T.T_ex[i+1] + T.T_ex[i]) / Δx^2
        end
        T.T_ex[2:end-1]  .=  T.T
    elseif alternative == 2
        # Alternative II ---                         
        # West
        T.T_ex[1]    =   (BC.type.W==:Dirichlet) * (2 * BC.val.W - T.T_ex[2]) + 
                                (BC.type.W==:Neumann) * (T.T_ex[2] - BC.val.W*Δx)
        # East
        T.T_ex[end]  =   (BC.type.W==:Dirichlet) * (2 * BC.val.E - T.T_ex[nc+1]) +
                                (BC.type.W==:Neumann) * (T.T_ex[nc+1] + BC.val.E*Δx)

        @. T.T[ind]  =   T.T_ex[ind+1] + κ * Δt * 
                                    (T.T_ex[ind + 2] - 2.0 * T.T_ex[ind+1] + T.T_ex[ind]) / Δx^2
        T.T_ex[2:end-1]  .=  T.T
    elseif alternative == 3            
        ForwardEuler1Dc!( T, κ, Δx, Δt, nc, BC )
    end
    # Calculate time ---
    Time    =   Time + Δt
    # Plot Solution ---
    p = plot(xc, T.T, xlabel="x [m]", ylabel="T [°C]", 
            title="Temperatur distribution after
            $(round(Time/day, digits=1)) days", 
            xlim=(0,L),ylim=(0, Tdike))    
    if save_fig == 1
        Plots.frame(anim)
    else
        display(p)
    end
end

We now need to create and save the animation:

In [ ]:
# Save Animation -------------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
else
    display(p)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
# ----------------------------------------------------------------------- #
b = time()
println(b-a)